<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/inria-25-alpha.png" style="display:inline"></span><br/>

# Extensions asynchrones du langage

# Accès http

In [ ]:
import time

urls = ["http://www.irs.gov/pub/irs-pdf/f1040.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040ez.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040es.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040sb.pdf"]

##### En version séquentielle

In [ ]:
import requests


beg = time.time()

for url in urls:
    req = requests.get(url)
    print(f"{url} returned {len(req.text)} chars")
    
print(f"duration = {time.time()-beg}s")

In [ ]:
import asyncio
import aiohttp

In [ ]:
async def fetch(url):
    async with aiohttp.ClientSession() as session:
        print(f"fetching {url}")
        async with session.get(url) as response:
            #print(f"{url} returned status {response.status}")
            raw = await response.read()
            print(f"{url} returned {len(raw)} bytes")

# Context managers asynchrones

* `__aenter__` et `__aexit__` : awaitables

* [défini dans PEP492](https://www.python.org/dev/peps/pep-0492/#asynchronous-context-managers-and-async-with)

In [ ]:
async def fetch_urls():
    await asyncio.gather(* (fetch(url) for url in urls))

In [ ]:
loop = asyncio.get_event_loop()
beg = time.time()
loop.run_until_complete(fetch_urls())
print(f"duration = {time.time()-beg}s")

# Itérations asynchrones

* boucle `async for` [PEP-492](https://www.python.org/dev/peps/pep-0492/#asynchronous-iterators-and-async-for)
* compréhensions asynchrones [PEP-530](https://www.python.org/dev/peps/pep-0530/)

In [ ]:
import asyncio

# une variante avec un itérateur asynchrone
async def fetch2(url, i):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            async for line in response.content:
                print(f'{i}', end='')
    return url

In [ ]:
asyncio.get_event_loop().run_until_complete(
    asyncio.gather(* (fetch2(url, i) for (i, url) in enumerate(urls))))

# Résumé (1)

* fonction coroutine `async def foo()`

* `foo()` renvoie un objet *coroutine*
  * il faut faire `await foo()`

**autorisé**

````
async def foo():
    await bar()
````

**pas autorisé** : *SyntaxError*

````
def foo():
    await bar()
````

# Résumé (2)

* Boucle d'événements 
  * `asyncio.get_event_loop()` 

* `async with`
* `async for`